In [1]:
import numpy as np
import tskit
import tszip
import time
import os
import gzip
import pandas as pd

In [2]:
sample_size = 1000
win_sizes = [100]

In [3]:
ts_gene_biallelic = tskit.load("../data/trees/ts_gene_biallelic_N10.trees")
ts_gene_biallelic

In [4]:
sites_pos = ts_gene_biallelic.sites_position
sites_pos

array([49000169., 49000607., 49000831., ..., 49998974., 49999218.,
       49999489.])

In [5]:
ldcalc = tskit.LdCalculator(ts_gene_biallelic)

In [10]:
df = pd.DataFrame()
target_sites_fwd=[]
target_sites_rev=[]
target_positions_fwd=[]
target_positions_rev=[]
r2_fwd=[]
r2_rev=[]

focal_sites=list(range(ts_gene_biallelic.num_sites))
df['focal_pos']=np.take(sites_pos, focal_sites).astype(int)
for focal_site in focal_sites:
    focal_r2_fwd = ldcalc.r2_array(focal_site, direction=tskit.FORWARD, max_mutations=2)
    focal_r2_rev = ldcalc.r2_array(focal_site, direction=tskit.REVERSE, max_mutations=2)
    r2_fwd.append(np.round(focal_r2_fwd,2))
    r2_rev.append(np.round(focal_r2_rev,2))
    focal_target_sites_fwd=np.arange(focal_site+1, focal_site+1+len(focal_r2_fwd))
    target_sites_fwd.append(focal_target_sites_fwd)
    target_positions_fwd.append(np.take(sites_pos, focal_target_sites_fwd).astype(int))
    focal_target_sites_rev=np.arange(focal_site-1,focal_site-1-len(focal_r2_rev),-1)
    target_sites_rev.append(focal_target_sites_rev)
    target_positions_rev.append(np.take(sites_pos,focal_target_sites_rev).astype(int))
    
df['target_sites_fwd']=target_sites_fwd
df['target_positions_fwd']=target_positions_fwd
df['r2_fwd']=r2_fwd
df['target_sites_rev']=target_sites_rev
df['target_positions_rev']=target_positions_rev
df['r2_rev']=r2_rev

df

,focal_pos,target_sites_fwd,target_positions_fwd,r2_fwd,target_sites_rev,target_positions_rev,r2_rev
0,49000169,"[1, 2]","[49000607, 49000831]","[0.11, 0.05]",[],[],[]
1,49000607,"[2, 3]","[49000831, 49002173]","[0.07, 0.0]",[0],[49000169],[0.11]
2,49000831,"[3, 4]","[49002173, 49002477]","[0.05, 0.02]","[1, 0]","[49000607, 49000169]","[0.07, 0.05]"
3,49002173,"[4, 5]","[49002477, 49002646]","[0.33, 0.33]","[2, 1]","[49000831, 49000607]","[0.05, 0.0]"
4,49002477,"[5, 6]","[49002646, 49002651]","[1.0, 1.0]","[3, 2]","[49002173, 49000831]","[0.33, 0.02]"
...,...,...,...,...,...,...,...
3258,49998548,"[3259, 3260]","[49998560, 49998974]","[0.01, 0.0]","[3257, 3256]","[49998044, 49997622]","[0.0, 0.01]"
3259,49998560,"[3260, 3261]","[49998974, 49999218]","[0.01, 0.05]","[3258, 3257]","[49998548, 49998044]","[0.01, 0.01]"
3260,49998974,"[3261, 3262]","[49999218, 49999489]","[0.01, 0.04]","[3259, 3258]","[49998560, 49998548]","[0.01, 0.0]"
3261,49999218,[3262],[49999489],[0.37],"[3260, 3259]","[49998974, 49998560]","[0.01, 0.05]"


In [23]:
df1 = pd.DataFrame()
focal_sites=[]
focal_positions=[]
target_sites=[]
target_positions=[]
r2=[]

for focal_site in range(ts_gene_biallelic.num_sites):
    r2_fwd = ldcalc.r2_array(focal_site, direction=tskit.FORWARD, max_mutations=20)
    r2_rev = ldcalc.r2_array(focal_site, direction=tskit.REVERSE, max_mutations=20)
    r2.append(np.round(r2_fwd,2))
    r2.append(np.round(r2_rev,2))
    focal_sites.append(np.repeat(focal_site,len(r2_fwd)+len(r2_rev)))
    focal_target_sites_fwd=np.arange(focal_site+1, focal_site+1+len(r2_fwd))
    target_sites.append(focal_target_sites_fwd)
    focal_target_sites_rev=np.arange(focal_site-1,focal_site-1-len(r2_rev),-1)
    target_sites.append(focal_target_sites_rev)
target_sites=np.concatenate(target_sites)
focal_sites=np.concatenate(focal_sites)
focal_positions=np.take(sites_pos, focal_sites).astype(int).flatten()
target_positions=np.take(sites_pos, target_sites).astype(int).flatten()
df1['focal_site']=focal_sites
df1['focal_position']=focal_positions
df1['target_site']=target_sites
df1['target_position']=target_positions
df1['r2']=np.concatenate(r2)
df1

,focal_site,focal_position,target_site,target_position,r2
0,0,49000169,1,49000607,0.11
1,0,49000169,2,49000831,0.05
2,0,49000169,3,49002173,0.01
3,0,49000169,4,49002477,0.03
4,0,49000169,5,49002646,0.03
...,...,...,...,...,...
130095,3262,49999489,3246,49990605,0.04
130096,3262,49999489,3245,49990147,0.26
130097,3262,49999489,3244,49988926,0.04
130098,3262,49999489,3243,49988764,0.04


In [25]:
df2=pd.read_pickle("../data/tskit_ld/r2_array_method/N10_w10.ld")
df2
df2

,focal_site,focal_position,target_site,target_position,r2
0,0,49000169,1,49000607,0.11
1,0,49000169,2,49000831,0.05
2,0,49000169,3,49002173,0.01
3,0,49000169,4,49002477,0.03
4,0,49000169,5,49002646,0.03
...,...,...,...,...,...
130095,3262,49999489,3246,49990605,0.04
130096,3262,49999489,3245,49990147,0.26
130097,3262,49999489,3244,49988926,0.04
130098,3262,49999489,3243,49988764,0.04
